In [1]:
 # Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import re

In [37]:
df = pd.read_csv(Path('all_transposed_frame_conversions.csv'))

# Review the columns
df.columns

Index(['play_uuid', 'x0', 'y0', 's0', 'a0', 'dis0', 'o0', 'dir0', 'x1', 'y1',
       ...
       'yardsToGo', 'preSnapHomeScore', 'preSnapVisitorScore',
       'offenseFormation', 'defendersInTheBox',
       'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
       'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'down'],
      dtype='object', length=184)

In [21]:
#df = df.drop(columns=['yardsToGo', 'preSnapHomeScore', 'preSnapVisitorScore',
#       'offenseFormation', 'defendersInTheBox',
#       'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
#       'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'down', 
#       'cumulativeOverall_D', 'cumulativeOverall_P',
#       'cumulativePerFormation_O', 'cumulativePerFormation_D',
#       'cumulativePerFormation_P', 'cumulativePerBoxCt_O',
#       'cumulativePerBoxCt_D', 'cumulativePerBoxCt_P', 'week', 'quarter', 'playType',
#       'ballDirection', 'cumulativeOverall_O'])
#df.columns

Index(['play_uuid', 'x0', 'y0', 's0', 'a0', 'dis0', 'o0', 'dir0', 'x1', 'y1',
       ...
       'a21', 'dis21', 'o21', 'dir21', 'x22', 'y22', 's22', 'a22', 'dis22',
       'converted'],
      dtype='object', length=161)

In [38]:
features_to_keep = ['converted', 'yardsToGo', 'preSnapHomeScore', 'preSnapVisitorScore',
       'offenseFormation', 'defendersInTheBox',
       'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
       'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'down', 
       'cumulativeOverall_D', 'cumulativeOverall_P',
       'cumulativePerFormation_O', 'cumulativePerFormation_D',
       'cumulativePerFormation_P', 'cumulativePerBoxCt_O',
       'cumulativePerBoxCt_D', 'cumulativePerBoxCt_P', 'week', 'quarter', 'playType',
       'ballDirection', 'cumulativeOverall_O']

df = df[features_to_keep]
df.columns

Index(['converted', 'yardsToGo', 'preSnapHomeScore', 'preSnapVisitorScore',
       'offenseFormation', 'defendersInTheBox',
       'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
       'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'down',
       'cumulativeOverall_D', 'cumulativeOverall_P',
       'cumulativePerFormation_O', 'cumulativePerFormation_D',
       'cumulativePerFormation_P', 'cumulativePerBoxCt_O',
       'cumulativePerBoxCt_D', 'cumulativePerBoxCt_P', 'week', 'quarter',
       'playType', 'ballDirection', 'cumulativeOverall_O'],
      dtype='object')

In [39]:
# Seperate the features, X,  from the target variable, y
# Keep meaningful features and target
#existing_features_to_keep = ['ballCarrierId', 'quarter', 'yardsToGo','preSnapHomeScore',
#       'preSnapVisitorScore', 'offenseFormation', 'defendersInTheBox',
#        'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded',
#        'visitorTeamWinProbilityAdded', 'down']
#
#created_features_to_keep = ['playType', 'ballDirection', 'cumulativeOverall_O', 
#        'cumulativeOverall_D', 'cumulativeOverall_P', 'cumulativePerFormation_O', 'cumulativePerFormation_D',
#        'cumulativePerFormation_P', 'cumulativePerBoxCt_O', 'cumulativePerBoxCt_D', 'cumulativePerBoxCt_P']


#existing_features_to_keep = ['yardsToGo', 'offenseFormation', 'defendersInTheBox']

#created_features_to_keep = ['playType']

y = df['converted']
X = df.drop(columns=['converted'])

#X = df[created_features_to_keep + existing_features_to_keep]
#X = df[created_features_to_keep]
#X = df[existing_features_to_keep]

In [40]:
X.head()

,yardsToGo,preSnapHomeScore,preSnapVisitorScore,offenseFormation,defendersInTheBox,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,down,...,cumulativePerFormation_D,cumulativePerFormation_P,cumulativePerBoxCt_O,cumulativePerBoxCt_D,cumulativePerBoxCt_P,week,quarter,playType,ballDirection,cumulativeOverall_O
0,10,0,0,I_FORM,6.0,0.399819,0.600181,-0.014850,0.014850,1,...,0.000000,0.000000,0.500000,0.500000,0.000000,1,1,not_pass,right,0.500000
1,10,0,10,SINGLEBACK,6.0,0.207237,0.792763,0.006345,-0.006345,1,...,1.000000,0.000000,0.142857,0.857143,0.000000,1,2,not_pass,right,0.200000
2,10,0,10,PISTOL,6.0,0.227212,0.772788,0.008701,-0.008701,1,...,0.000000,0.000000,0.222222,0.777778,0.000000,1,2,not_pass,left,0.250000
3,10,0,10,SINGLEBACK,6.0,0.235201,0.764799,0.027923,-0.027923,1,...,1.000000,0.000000,0.181818,0.818182,0.000000,1,2,not_pass,left,0.250000
4,7,0,10,SINGLEBACK,6.0,0.263124,0.736876,0.000100,-0.000100,1,...,0.833333,0.333333,0.250000,0.750000,0.166667,1,2,not_pass,middle,0.294118


In [41]:
X = pd.get_dummies(X)
X.head()

,yardsToGo,preSnapHomeScore,preSnapVisitorScore,defendersInTheBox,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,down,cumulativeOverall_D,...,offenseFormation_PISTOL,offenseFormation_SHOTGUN,offenseFormation_SINGLEBACK,offenseFormation_WILDCAT,playType_not_pass,playType_pass,ballDirection_left,ballDirection_middle,ballDirection_none,ballDirection_right
0,10,0,0,6.0,0.399819,0.600181,-0.014850,0.014850,1,0.500000,...,False,False,False,False,True,False,False,False,False,True
1,10,0,10,6.0,0.207237,0.792763,0.006345,-0.006345,1,0.800000,...,False,False,True,False,True,False,False,False,False,True
2,10,0,10,6.0,0.227212,0.772788,0.008701,-0.008701,1,0.750000,...,True,False,False,False,True,False,True,False,False,False
3,10,0,10,6.0,0.235201,0.764799,0.027923,-0.027923,1,0.750000,...,False,False,True,False,True,False,True,False,False,False
4,7,0,10,6.0,0.263124,0.736876,0.000100,-0.000100,1,0.705882,...,False,False,True,False,True,False,False,True,False,False


In [42]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [43]:
 # Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [44]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3
knn = KNeighborsClassifier(n_neighbors=10)

In [45]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=10)

In [46]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [47]:
 # Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.97      0.87      1186
           1       0.72      0.21      0.32       389

    accuracy                           0.78      1575
   macro avg       0.76      0.59      0.60      1575
weighted avg       0.77      0.78      0.74      1575

